In [1]:
import json
import pickle
import cv2
import numpy as np

# !pip install import-ipynb
import import_ipynb
from stereo_camera_calibration import *

importing Jupyter notebook from stereo_camera_calibration.ipynb


In [4]:
def read_keypoints(path):
    
    with open(path) as input_file:
        all_data = json.load(input_file)['captures'][2:]
    left, right = [], []
    left_frame_keypoints = []
    right_frame_keypoints = []
    
    for i in range(len(all_data)):
        if i % 2 == 0:
            left.append(all_data[i])
        else:
            right.append(all_data[i])
            
    for i in range(len(left)):
        left_frame_keypoints.append(get_frame_points(left[i]['annotations'][3]['values'][0]['keypoints']))
    for i in range(len(right)):
        right_frame_keypoints.append(get_frame_points(right[i]['annotations'][3]['values'][0]['keypoints']))
        
    print("find {} frames from left camera.".format(len(left_frame_keypoints)))
    for i in range(len(left_frame_keypoints)):
        print("find {} key points in left frame {}.".format(len(left_frame_keypoints[i]), i))
    print("=====================================================")
    print("find {} frames from right camera.".format(len(right_frame_keypoints)))
    for i in range(len(right_frame_keypoints)):
        print("find {} key points in right frame {}.".format(len(right_frame_keypoints[i]), i))
    
    keypoint_data = {"left": left_frame_keypoints, "right": right_frame_keypoints}
    return keypoint_data

def get_frame_points(frame):
    result = []
    for i in range(len(frame)):
        if frame[i]['x'] != 0 and frame[i]['y'] != 0:
            result.append((frame[i]['x'], frame[i]['y']))
    return result

In [3]:
def read_cal_model(path):
    with open(path, "rb") as input_file:
        cal_data = pickle.load(input_file)
    return cal_data

In [16]:
def triangulate(cal_data, keypoint_data):
    R1, R2, P1, P2, Q, roi1, roi2 = cv2.stereoRectify(
        cal_data.camera_model["M1"],
        cal_data.camera_model["dist1"],
        cal_data.camera_model["M2"],
        cal_data.camera_model["dist2"],
        (1024, 1024),
        cal_data.camera_model["R"],
        cal_data.camera_model["T"]
    )
    
#     print("P1", P1)
#     print("P2", P2)
    left_frame_keypoints = keypoint_data["left"]
    right_frame_keypoints = keypoint_data["right"]

    # print(P1)

    # P1 = np.array([[2.605089, 0.0, 0.0], [0.0, 2.605089, 0.0], [0.0, 0.0, -1.03045678]])
    # P2 = np.array([[2.605089, 0.0, 0.0], [0.0, 2.605089, 0.0], [0.0, 0.0, -1.03045678]])
    
    # iterate on number of frames
    for i in range(len(left_frame_keypoints)):
        # iterate on each pair of key point per frame
        for j in range(len(left_frame_keypoints[i])):
            triang_point = cv2.triangulatePoints(P1, P2, left_frame_keypoints[i][j], right_frame_keypoints[i][j])
            # print(triang_point)
            euclidian = triang_point / triang_point[3]
            euclidian = np.asarray(euclidian, dtype=np.float32)
            print(f"Euclidian Co-ordinate: X:{euclidian[0]} Y:{euclidian[1]} Z:{euclidian[2]}")

In [17]:
if __name__ == "__main__":
    keypoint_data = read_keypoints("./test_frames/TestJsons/captures_000.json")
    cal_data = read_cal_model("./camera_model.pickle")
    triangulate(cal_data, keypoint_data)

find 2 frames from left camera.
find 10 key points in left frame 0.
find 10 key points in left frame 1.
find 2 frames from right camera.
find 10 key points in right frame 0.
find 10 key points in right frame 1.
Euclidian Co-ordinate: X:[-3.343412] Y:[-2.8360858] Z:[1.3446834]
Euclidian Co-ordinate: X:[-3.3425992] Y:[-2.8308406] Z:[1.3445036]
Euclidian Co-ordinate: X:[-3.3419876] Y:[-2.826885] Z:[1.3443686]
Euclidian Co-ordinate: X:[-3.3413742] Y:[-2.8229322] Z:[1.3442327]
Euclidian Co-ordinate: X:[-3.3407943] Y:[-2.819181] Z:[1.3441046]
Euclidian Co-ordinate: X:[-3.3383057] Y:[-2.8321972] Z:[1.3426058]
Euclidian Co-ordinate: X:[-3.3374908] Y:[-2.8269343] Z:[1.3424257]
Euclidian Co-ordinate: X:[-3.3368797] Y:[-2.822987] Z:[1.3422906]
Euclidian Co-ordinate: X:[-3.3362687] Y:[-2.81904] Z:[1.3421556]
Euclidian Co-ordinate: X:[-3.3356855] Y:[-2.8152742] Z:[1.3420266]
Euclidian Co-ordinate: X:[-3.8396304] Y:[-2.8297226] Z:[1.5593957]
Euclidian Co-ordinate: X:[-3.8352385] Y:[-2.8312447] Z:[1.

In [9]:
    cal_data = read_cal_model("./camera_model.pickle")
    print(np.linalg.norm(cal_data.camera_model["T"]))

0.33983423853199174
